In [1]:
import numpy as np
%matplotlib inline
from devito import Grid, Function, TimeFunction, SparseTimeFunction
from devito import solve, Eq, Operator, ConditionalDimension, Buffer, Constant
from examples.seismic import Model, plot_velocity, TimeAxis, RickerSource, Receiver
import matplotlib.pyplot as plt

## Acoustic Wave equation

$$ \frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2}  = \rho \nabla \frac{1}{\rho} . \nabla u + s$$

With damping and setting $\rho=1/b$ , we have 
$$ \frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t}  = \frac{1}{b} \nabla b . \nabla u + s$$

In one dimension, 

$$ \frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t}  = \frac{1}{b} \frac{\partial}{\partial x} \left(b . \frac{\partial u}{\partial x}\right) + s$$

$$
= \frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t}  = \frac{1}{b}\frac{d b}{d x}\frac{\partial u}{\partial x} + \frac{\partial^2 u}{\partial x^2} + s
$$
with constant density, this becomes
$$
\frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t}  = \frac{\partial^2 u}{\partial x^2} + s
$$

With 4th order approximation, 

$$
\frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t} = \frac{1}{b} \frac{\partial}{\partial x} \left(b . \frac{\partial u}{\partial x}\right) + \frac{\delta t^2}{12} \frac{1}{b} \frac{\partial }{\partial x} \left(b \frac{\partial }{\partial x} \left( \frac{v^2}{b} \frac{\partial }{\partial x} b \frac{\partial u}{\partial x} \right) \right) + s
$$

In [2]:
dim = 1
shape = (201, )
spacing = (50, )

so, to = 2, 2
v_water = 1.500
v_rock = 2.000 # Could be upto 4.000 (salt closer to 5.000)
nbl = 10 # Number of boundary layers

origin = (0, )
xmax = (shape[0]-1 + 2*nbl) * spacing[0]
comp_xmax = (shape[0]-1) * spacing[0]
n_src, n_recv = 1, 1
src_coords = np.array((0.2 * comp_xmax, ))
recv_coords = np.array((0.8 * comp_xmax, ))

v_initial = np.ones(shape) * v_water
b_initial = np.ones(shape)
model = Model(origin, spacing, shape, so, v_initial, nbl=nbl, bcs='damp', b=b_initial)
dt = model.critical_dt
print(dt)

s = Constant(name='s')
s.data = model.critical_dt
t0 = 0
tn = 1e4
time_range = TimeAxis(start=t0, stop=tn, step=s.data)
f0 = 0.001

src = RickerSource(name='src', grid=model.grid, f0=f0,
                   npoint=n_src, time_range=time_range)
src.coordinates.data[:] = src_coords

rec = Receiver(name='rec', grid=model.grid, npoint=n_recv, time_range=time_range)
rec.coordinates.data[:] = recv_coords
u = TimeFunction(name='u', grid=model.grid, space_order=so, time_order=to, save=Buffer(time_range.num))

t = model.grid.time_dim
x = model.grid.dimensions

pde = model.m * u.dt2 + model.damp * u.dt - u.dx2
pde

Operator `initdamp` ran in 0.01 s
Operator `pad_vp` ran in 0.01 s
Operator `pad_b` ran in 0.01 s


28.87


damp(x)*Derivative(u(t, x), t) - Derivative(u(t, x), (x, 2)) + Derivative(u(t, x), (t, 2))/vp(x)**2

In [3]:
buoyant_pde = model.m * u.dt2 + model.damp * u.dt - 1/model.b * (model.b * u.dxc).dxc
buoyant_pde

damp(x)*Derivative(u(t, x), t) + Derivative(u(t, x), (t, 2))/vp(x)**2 - Derivative(b(x)*Derivative(u(t, x), x), x)/b(x)

In [4]:
stencil = Eq(u.forward, solve(pde, u.forward))
src_term = src.inject(field=u.forward, expr=src * s**2 / model.m)
rec_term = rec.interpolate(expr=u.forward)
stencil2 = Eq(u.forward, solve(buoyant_pde, u.forward))
stencil2

Eq(u(t + dt, x), (-(-2.0*u(t, x)/dt**2 + u(t - dt, x)/dt**2)/vp(x)**2 + Derivative(b(x)*Derivative(u(t, x), x), x)/b(x) + damp(x)*u(t, x)/dt)/(damp(x)/dt + 1/(dt**2*vp(x)**2)))

In [72]:
# op1 = Operator(stencil + src_term + rec_term)
op2 = Operator(stencil2 + src_term + rec_term)

In [73]:
# for ci in op1.cinterface():
#     print(ci)

In [74]:
print(op2.ccode)

#define _POSIX_C_SOURCE 200809L
#define START_TIMER(S) struct timeval start_ ## S , end_ ## S ; gettimeofday(&start_ ## S , NULL);
#define STOP_TIMER(S,T) gettimeofday(&end_ ## S, NULL); T->S += (double)(end_ ## S .tv_sec-start_ ## S.tv_sec)+(double)(end_ ## S .tv_usec-start_ ## S .tv_usec)/1000000;

#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct dataobj
{
  void *restrict data;
  unsigned long * size;
  unsigned long * npsize;
  unsigned long * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
  double section2;
} ;


int Kernel(struct dataobj *restrict b_vec, struct dataobj *restrict damp_vec, struct dataobj *restrict rec_vec, struct dataobj *restrict rec_coords_vec, const float s, struct dataobj *restrict src_vec, struct dataobj *restrict src_coords_vec, struct dataobj *restrict u_vec, struct dataobj *restrict vp_vec, const int x_M, const int x_m, const flo

$$
\frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t} = \frac{1}{b} \frac{\partial}{\partial x} \left(b . \frac{\partial u}{\partial x}\right) + \frac{\delta t^2}{12} \frac{1}{b} \frac{\partial }{\partial x} \left(b \frac{\partial }{\partial x} \left( \frac{v^2}{b} \frac{\partial }{\partial x} b \frac{\partial u}{\partial x} \right) \right) + s
$$

In [12]:
pde = model.m * u.dt2 - 0.5*(u.dxr * model.b.dxr + u.dxl * model.b.dxl)
pde

damp(x)*Derivative(u(t, x), t) - 0.5*Derivative(b(x), x)*Derivative(u(t, x), x) - 0.5*Derivative(b(x), x)*Derivative(u(t, x), x) + Derivative(u(t, x), (t, 2))/vp(x)**2

In [13]:
stencil3 = Eq(u.forward, solve(pde, u.forward))
stencil3

Eq(u(t + dt, x), (-(-2.0*u(t, x)/dt**2 + u(t - dt, x)/dt**2)/vp(x)**2 + 0.5*Derivative(b(x), x)*Derivative(u(t, x), x) + 0.5*Derivative(b(x), x)*Derivative(u(t, x), x) + damp(x)*u(t, x)/dt)/(damp(x)/dt + 1/(dt**2*vp(x)**2)))

In [14]:
op3 = Operator(stencil3 + src_term + rec_term)
print(op3.ccode)

#define _POSIX_C_SOURCE 200809L
#define START_TIMER(S) struct timeval start_ ## S , end_ ## S ; gettimeofday(&start_ ## S , NULL);
#define STOP_TIMER(S,T) gettimeofday(&end_ ## S, NULL); T->S += (double)(end_ ## S .tv_sec-start_ ## S.tv_sec)+(double)(end_ ## S .tv_usec-start_ ## S .tv_usec)/1000000;

#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct dataobj
{
  void *restrict data;
  unsigned long * size;
  unsigned long * npsize;
  unsigned long * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
  double section2;
} ;


int Kernel(struct dataobj *restrict b_vec, struct dataobj *restrict damp_vec, struct dataobj *restrict rec_vec, struct dataobj *restrict rec_coords_vec, const float s, struct dataobj *restrict src_vec, struct dataobj *restrict src_coords_vec, struct dataobj *restrict u_vec, struct dataobj *restrict vp_vec, const int x_M, const int x_m, const flo

$$
\frac{1}{v^{2}} \frac{\partial^2 u}{\partial t^2} + \eta \frac{\partial u}{\partial t} = \frac{1}{b} \frac{\partial}{\partial x} \left(b . \frac{\partial u}{\partial x}\right) + \frac{\delta t^2}{12} \frac{1}{b} \frac{\partial }{\partial x} \left(b \frac{\partial }{\partial x} \left( \frac{v^2}{b} \frac{\partial }{\partial x} b \frac{\partial u}{\partial x} \right) \right) + s
$$

In [52]:
s = Constant(name='s')
s.data = model.critical_dt
fo_pde = model.m * u.dt2 + model.damp*u.dt - 1/model.b * (model.b * u.dxc).dxc + \
s**2 / 12 * 1/model.b * (model.b * (1/model.m * 1/model.b * (model.b * u.dxc).dxc).dxc).dxc
fo_pde

s**2*Derivative(b(x)*Derivative(vp(x)**2*Derivative(b(x)*Derivative(u(t, x), x), x)/b(x), x), x)/(12*b(x)) + damp(x)*Derivative(u(t, x), t) + Derivative(u(t, x), (t, 2))/vp(x)**2 - Derivative(b(x)*Derivative(u(t, x), x), x)/b(x)

In [53]:
stencil4 = Eq(u.forward, solve(fo_pde, u.forward))
stencil4

Eq(u(t + dt, x), (-s**2*Derivative(b(x)*Derivative(vp(x)**2*Derivative(b(x)*Derivative(u(t, x), x), x)/b(x), x), x)/(12*b(x)) - (-2.0*u(t, x)/dt**2 + u(t - dt, x)/dt**2)/vp(x)**2 + Derivative(b(x)*Derivative(u(t, x), x), x)/b(x) + damp(x)*u(t, x)/dt)/(damp(x)/dt + 1/(dt**2*vp(x)**2)))

In [51]:
op4 = Operator(stencil4 + src_term + rec_term)
print(op4.ccode)

#define _POSIX_C_SOURCE 200809L
#define START_TIMER(S) struct timeval start_ ## S , end_ ## S ; gettimeofday(&start_ ## S , NULL);
#define STOP_TIMER(S,T) gettimeofday(&end_ ## S, NULL); T->S += (double)(end_ ## S .tv_sec-start_ ## S.tv_sec)+(double)(end_ ## S .tv_usec-start_ ## S .tv_usec)/1000000;

#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct dataobj
{
  void *restrict data;
  unsigned long * size;
  unsigned long * npsize;
  unsigned long * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
  double section2;
} ;


int Kernel(struct dataobj *restrict b_vec, struct dataobj *restrict damp_vec, struct dataobj *restrict rec_vec, struct dataobj *restrict rec_coords_vec, const float s, const float s, struct dataobj *restrict src_vec, struct dataobj *restrict src_coords_vec, struct dataobj *restrict u_vec, struct dataobj *restrict vp_vec, const int x_M, const int